In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.optimize as opt
%matplotlib inline

In [2]:
dataset = pd.read_csv('housing.csv', sep = ";", header = None).astype(str)
dataset.head()

,0
0,0.00632 18.00 2.310 0 0.5380 6.5750 65...
1,0.02731 0.00 7.070 0 0.4690 6.4210 78...
2,0.02729 0.00 7.070 0 0.4690 7.1850 61...
3,0.03237 0.00 2.180 0 0.4580 6.9980 45...
4,0.06905 0.00 2.180 0 0.4580 7.1470 54...


# Data Cleaning

In [3]:
def visualize(data, column_1, column_2):
    fig, ax = plt.subplots()
    ax.scatter(data.iloc[:, column_1].values, data.iloc[:, column_2].values)
    ax.set_title('Diabetes Factor')
    ax.set_xlabel(dataset.keys()[column_1])
    ax.set_ylabel(dataset.keys()[column_2])
    
visualize(dataset, index,index_y)

NameError: name 'index' is not defined

In [ ]:
# Since there are some spaces in the data, we have to remove the spaces first before converting it back to float
dataset[1][70:]

In [ ]:
for column in dataset:
    for row in range(len(dataset)):
        dataset[column][row] = dataset[column][row].replace(" ", "")

In [ ]:
dataset = dataset.astype(float)

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.dropna()

In [ ]:
dataset.describe()

# Handling Outlier and Scaling the dataset

In [ ]:
import seaborn as sns
outliers = sns.boxplot(data=dataset, orient="h", palette="Set2")

Based on the observation above we knew that there are few column that has outlier, therefore we have to remove all of the outlier from each column

In [ ]:
Q1 = dataset.quantile(0.25)
Q3 = dataset.quantile(0.75)
IQR = Q3-Q1
print(IQR)

In [ ]:
# Remove Outlier from the dataset
dataset = dataset[~((dataset < (Q1 - 1.5 * IQR)) |(dataset > (Q3 + 1.5 * IQR))).any(axis=1)]
dataset.shape

In [ ]:
outliers = sns.boxplot(data=dataset, orient="h", palette="Set2")

# Min Max Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(dataset)
dataset = scaler.transform(dataset)
dataset = pd.DataFrame(dataset)

In [ ]:
dataset.describe()

Each feature column has a different range, not all of them are uniform. We have to do normalization technique, since we have removed all of the outlier, we can use min max scaller as the normalization technique

# Data Preparation

In [ ]:
dataset_x     = dataset.loc[:,dataset.columns != 13]
dataset_y     = dataset[13].values

In [ ]:
dataset_x.describe()

# Feature Selection

In [ ]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

fs = SelectKBest(score_func=f_regression, k=5)
fit = fs.fit(dataset_x,dataset_y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(dataset.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Columns','F_Score'] 

print(featureScores.nlargest(12,'F_Score'))  

Since the vast majority of the features only have a small f score, we only need the top 5 highest score as our main features.

In [ ]:
cleaned_dataset = dataset[[12,5,6,4,9,13]]

In [ ]:
cleaned_dataset.describe()

# Splitting the dataset

In [ ]:
train_dataset = cleaned_dataset.sample(frac=0.8)
test_dataset = cleaned_dataset.drop(train_dataset.index)

train_x = train_dataset.loc[:,train_dataset.columns != 13] 
test_x = test_dataset.loc[:,test_dataset.columns != 13]

train_y = train_dataset[13].values
test_y = test_dataset[13].values

# Multivariate Linear Regression

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error

regressor = linear_model.LinearRegression()
regressor.fit(train_x, train_y)

dataset_y_predicted = regressor.predict(test_x)

In [ ]:
print('Coefficients: \n', regressor.coef_)
print('Mean squared error: %.2f' % 
      mean_squared_error(test_y, dataset_y_predicted))

In [ ]:
plt.figure()
plt.scatter(x=list(range(0, len(test_y))), y=test_y, color='red')         
plt.scatter(x=list(range(0, len(dataset_y_predicted))), y=dataset_y_predicted, color='black')
plt.show()